## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-02-06-31-35 +0000,nyt,"Fire in Hermosillo, Mexico, Kills at Least 23 ...",https://www.nytimes.com/2025/11/01/world/ameri...
1,2025-11-02-05-50-15 +0000,48hills,Queer activists ‘Scare the State’ at ICE prote...,https://48hills.org/2025/11/queer-activists-sc...
2,2025-11-02-05-29-22 +0000,bbc,Tracking the ICE tactics used in Trump's mass ...,https://www.bbc.com/news/videos/c6299nrj76yo?a...
3,2025-11-02-05-06-26 +0000,nyt,He Stayed in Belarus for His Imprisoned Wife. ...,https://www.nytimes.com/2025/11/01/world/europ...
4,2025-11-02-04-54-10 +0000,bbc,Dodgers seal back-to-back World Series titles,https://www.bbc.com/sport/baseball/articles/c4...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,25
83,election,10
103,mamdani,8
20,he,8
122,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2025-11-01-08-43-09 +0000,nyt,Carney Says He Apologized to Trump Over Ad Rev...,https://www.nytimes.com/2025/11/01/world/canad...,53
118,2025-11-01-14-00-00 +0000,wsj,Canadian PM Carney Says He Apologized to Trump...,https://www.wsj.com/world/canadian-pm-carney-s...,52
63,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...,50
74,2025-11-01-20-44-56 +0000,bbc,US judges say Trump administration must contin...,https://www.bbc.com/news/articles/cr433x9zqq4o...,50
69,2025-11-01-20-55-31 +0000,nypost,House Republicans exploring ways to prevent Ma...,https://nypost.com/2025/11/01/us-news/house-re...,50


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,53,2025-11-01-08-43-09 +0000,nyt,Carney Says He Apologized to Trump Over Ad Rev...,https://www.nytimes.com/2025/11/01/world/canad...
69,42,2025-11-01-20-55-31 +0000,nypost,House Republicans exploring ways to prevent Ma...,https://nypost.com/2025/11/01/us-news/house-re...
68,34,2025-11-01-21-04-33 +0000,nypost,NJ Dem chair admits ‘New Jersey is a battlegro...,https://nypost.com/2025/11/01/us-news/nj-dem-b...
65,29,2025-11-01-21-27-14 +0000,latimes,"As SNAP benefits lapse, thousands show up to S...",https://www.latimes.com/california/story/2025-...
17,27,2025-11-02-03-25-50 +0000,nypost,Hero old man saves girl from knife-wielding ma...,https://nypost.com/2025/11/01/world-news/uk-tr...
44,26,2025-11-01-23-39-02 +0000,bbc,300 million tourists just visited China's stun...,https://www.bbc.com/news/articles/c8x1rnnd5gjo...
31,25,2025-11-02-01-28-51 +0000,nypost,Obama calls Zohran Mamdani and offers to be ‘s...,https://nypost.com/2025/11/01/us-news/obama-ha...
92,25,2025-11-01-18-25-24 +0000,nypost,Nicki Minaj sends lefty fans into meltdown aft...,https://nypost.com/2025/11/01/us-news/nicki-mi...
93,24,2025-11-01-18-25-22 +0000,nyt,Two More Suspects Charged in Louvre Heist Case,https://www.nytimes.com/2025/11/01/world/europ...
55,24,2025-11-01-22-52-18 +0000,nypost,NYPD detective removed from Israel post after ...,https://nypost.com/2025/11/01/us-news/israel-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
